## LAB-25 抽樣
1. 簡單隨機抽樣（simple random sampling）
2. 系統抽樣（systematic sampling）
3. 分層抽樣（stratified sampling）
4. 整群抽樣（cluster sampling）

In [1]:
# 檢查套件是否已安裝
!conda list "pandas|numpy|tqdm|openpyxl"

# packages in environment at /opt/homebrew/Caskroom/miniforge/base/envs/lab:
#
# Name                    Version                   Build  Channel
numpy                     2.2.4           py313h41a2e72_0    conda-forge
openpyxl                  3.1.5           py313h90caf49_1    conda-forge
pandas                    2.2.3           py313h47b39a6_1    conda-forge
tqdm                      4.67.1             pyhd8ed1ab_1    conda-forge


In [2]:
import random
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm 
import csv
#from rich import print 

In [3]:
# 使用 lab-24 資料清理的結果
csvf = "lab-24.csv"

df = pd.read_csv(csvf)

In [4]:
# 資料筆數
df.shape

(18282, 11)

In [5]:
# 檢視資料
df.head(5)

,統一編號,公司名稱,負責人,公司地址,資本總額,實收資本額,在境內營運資金,產製日期,縣市,狀態,類別
0,1200,形韻科技股份有限公司,林O民,新竹市東區中華路二段198號1樓及地下一樓,1000000.0,1000000.0,0.0,2025-03-01,新竹市,設立,公司登記
1,2799,鑫晶生物科技有限公司,陳O英,臺北市大同區市民大道1段209號10樓,1000000.0,0.0,0.0,2025-03-01,臺北市,設立,公司登記
2,4810,道原智慧服務有限公司,廖O奇,新北市五股區中興路1段162號（2樓）,2000000.0,0.0,0.0,2025-03-01,新北市,設立,公司登記
3,4945,顯樂文化事業有限公司,黃O宇,花蓮縣新城鄉新秀村光復路497號2樓之1,100000.0,0.0,0.0,2025-03-01,花蓮縣,設立,公司登記
4,5938,瑞盛人力資源管理顧問有限公司,陳O瑞,臺北市信義區信義路4段415號5樓之4,100000.0,0.0,0.0,2025-03-01,臺北市,設立,公司登記


In [6]:
# 欄位結構
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18282 entries, 0 to 18281
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   統一編號     18282 non-null  int64  
 1   公司名稱     18282 non-null  object 
 2   負責人      11172 non-null  object 
 3   公司地址     18282 non-null  object 
 4   資本總額     16045 non-null  float64
 5   實收資本額    16045 non-null  float64
 6   在境內營運資金  16045 non-null  float64
 7   產製日期     16045 non-null  object 
 8   縣市       18282 non-null  object 
 9   狀態       18282 non-null  object 
 10  類別       18282 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 1.5+ MB


### 資料處理
身分證號左補零

In [7]:
df["統一編號"] = df["統一編號"].astype(str).str.zfill(8)

In [8]:
# 若是 float 需轉換為 int 
cols = ["資本總額","實收資本額","在境內營運資金"]

for c in cols:
    if df[c].dtype == 'float64':
        df[c] = df[c].fillna(0).astype(int)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18282 entries, 0 to 18281
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   統一編號     18282 non-null  object
 1   公司名稱     18282 non-null  object
 2   負責人      11172 non-null  object
 3   公司地址     18282 non-null  object
 4   資本總額     18282 non-null  int64 
 5   實收資本額    18282 non-null  int64 
 6   在境內營運資金  18282 non-null  int64 
 7   產製日期     16045 non-null  object
 8   縣市       18282 non-null  object
 9   狀態       18282 non-null  object
 10  類別       18282 non-null  object
dtypes: int64(3), object(8)
memory usage: 1.5+ MB


### 設定母體：「狀態」= "設立"

In [10]:
# 只查核目前有效的單位
df1 = df[df["狀態"] == "設立"]

# 母體筆數
M = df1.shape[0]
print(f"原資料筆數：{df.shape[0]:,d} -> 母體筆數：{M:,d}")

原資料筆數：18,282 -> 母體筆數：12,138


### 簡單隨機取樣（simple random sampling）<a id="01"></a>

從母體N個單位中隨機地抽取n個單位作為樣本，使得每一個容量為樣本都有相同的機率被抽中。特點是：每個樣本單位被抽中的機率相等，樣本的每個單位完全獨立，彼此間無一定的關聯性和排斥性。簡單隨 機抽樣是其它各種抽樣形式的基礎。通常只是在母體單位之間差異程度較小和數目較少時，才採用這種方法。

In [11]:
# 方法一：固定筆數
samp_cnt = 120
df2 = df1.sample(n=samp_cnt)


In [12]:
# 方法二：百分比
samp_rate = 0.01    # 1%
df3 = df1.sample(frac=samp_rate)


In [13]:
# 顯示抽樣的結果
# 先統計 母體的「縣市」
df1_sum = df1["縣市"].value_counts().reset_index(name="件數")
df1_sum 

,縣市,件數
0,臺北市,3623
1,新北市,2241
2,臺中市,1906
3,高雄市,1384
4,桃園市,850
5,臺南市,538
6,彰化縣,217
7,新竹市,173
8,新竹縣,171
9,屏東縣,146


In [14]:
# 統計抽樣結果
df2_sum = df2["縣市"].value_counts().reset_index(name="抽樣")
df3_sum = df3["縣市"].value_counts().reset_index(name="抽樣")

# 使用 merge 合併
df1_sum = pd.merge(df1_sum, df2_sum, on="縣市", how='left')
df1_sum = pd.merge(df1_sum, df3_sum, on="縣市", suffixes=('1', '2'),how='left')

df1_sum

,縣市,件數,抽樣1,抽樣2
0,臺北市,3623,33.0,37.0
1,新北市,2241,16.0,20.0
2,臺中市,1906,17.0,16.0
3,高雄市,1384,13.0,14.0
4,桃園市,850,13.0,9.0
5,臺南市,538,11.0,11.0
6,彰化縣,217,3.0,1.0
7,新竹市,173,1.0,NaN
8,新竹縣,171,2.0,1.0
9,屏東縣,146,1.0,2.0


In [15]:
df1_sum = df1_sum.fillna(0)
df1_sum

,縣市,件數,抽樣1,抽樣2
0,臺北市,3623,33.0,37.0
1,新北市,2241,16.0,20.0
2,臺中市,1906,17.0,16.0
3,高雄市,1384,13.0,14.0
4,桃園市,850,13.0,9.0
5,臺南市,538,11.0,11.0
6,彰化縣,217,3.0,1.0
7,新竹市,173,1.0,0.0
8,新竹縣,171,2.0,1.0
9,屏東縣,146,1.0,2.0


In [16]:
# 儲存抽樣結果
# 儲存為 CSV

# 母體
csvf = "lab-25-0.csv"
df1.to_csv(csvf, index=False, encoding='utf-8-sig', quoting = csv.QUOTE_NONNUMERIC)
# 抽樣 1
csvf = "lab-25-1.csv"
df2.to_csv(csvf, index=False, encoding='utf-8-sig', quoting = csv.QUOTE_NONNUMERIC)
# 抽樣 2
csvf = "lab-25-2.csv"
df3.to_csv(csvf, index=False, encoding='utf-8-sig', quoting = csv.QUOTE_NONNUMERIC)


In [17]:
# 儲存 Excel 
xlsx = "lab-25-簡單隨機抽樣.xlsx"

with pd.ExcelWriter(xlsx) as writer:
    df.to_excel(writer, sheet_name='第三方支付', index=False)
    df1.to_excel(writer, sheet_name='母體', index=False)
    df2.to_excel(writer, sheet_name='抽樣1', index=False)
    df3.to_excel(writer, sheet_name='抽樣2', index=False)
    df1_sum.to_excel(writer, sheet_name='抽樣統計', index=False)

    

In [18]:
# 另一種儲存方式
df4 = df.copy()
df4['母體'] = df4.index.isin(df1.index)


In [19]:
df4['母體'].value_counts()

母體
True     12138
False     6144
Name: count, dtype: int64

In [20]:
# 抽樣1
df4['抽樣1'] = df4.index.isin(df2.index)

# 抽樣2
df4['抽樣2'] = df4.index.isin(df3.index)


In [21]:
print(f"母體： {df4["母體"].sum():10,d}")
print(f"抽樣1：{df4["抽樣1"].sum():10,d}")
print(f"抽樣2：{df4["抽樣2"].sum():10,d}")


母體：     12,138
抽樣1：       120
抽樣2：       121


In [22]:
# 將 BOOL 轉換為 Y | 空白
df4['母體'] = df4['母體'].apply(lambda x: 'Y' if x else '')
df4['抽樣1'] = df4['抽樣1'].apply(lambda x: 'Y' if x else '')
df4['抽樣2'] = df4['抽樣2'].apply(lambda x: 'Y' if x else '')

In [23]:
# 儲存 Excel 
xlsx = "lab-25-簡單隨機抽樣-2.xlsx"

with pd.ExcelWriter(xlsx) as writer:
    df4.to_excel(writer, sheet_name='簡單亂數抽樣', index=False)

### 系統抽樣（systematic sampling）<a id="02"></a>

也稱等距抽樣。將母體中的所有單位按一定順序排列，在規定的範圍內隨機地抽取一個單位作為初始單位，然後按事先規定好的規則確定其他樣本單位。先從數字1到k之間隨機抽取一個數字r作為初始單位，以後依次取r+k、r+2k……等單位。這種方法操作簡便，可提高估計的精度。


1. 母體  N 初始位置  r 間隔  k
2. 間隔(k) = 10 (參數)
3. 初始位置(r) = 1-k 之亂數
4. 每隔 k 抽 第 r 個位置，直到抽滿 N

> 同序列 [r, r+k, r+2k, r+3k, .... | < N]


In [24]:
# 複製資料集
df_sys = df1.copy()

df_sys = df_sys.reset_index(drop=True)
df_sys.head(10)

,統一編號,公司名稱,負責人,公司地址,資本總額,實收資本額,在境內營運資金,產製日期,縣市,狀態,類別
0,00001200,形韻科技股份有限公司,林O民,新竹市東區中華路二段198號1樓及地下一樓,1000000,1000000,0,2025-03-01,新竹市,設立,公司登記
1,00002799,鑫晶生物科技有限公司,陳O英,臺北市大同區市民大道1段209號10樓,1000000,0,0,2025-03-01,臺北市,設立,公司登記
2,00004810,道原智慧服務有限公司,廖O奇,新北市五股區中興路1段162號（2樓）,2000000,0,0,2025-03-01,新北市,設立,公司登記
3,00004945,顯樂文化事業有限公司,黃O宇,花蓮縣新城鄉新秀村光復路497號2樓之1,100000,0,0,2025-03-01,花蓮縣,設立,公司登記
4,00005938,瑞盛人力資源管理顧問有限公司,陳O瑞,臺北市信義區信義路4段415號5樓之4,100000,0,0,2025-03-01,臺北市,設立,公司登記
5,00006438,璟朝開發有限公司,李O煌,桃園市中壢區正光街175號(2樓),1000000,0,0,2025-03-01,桃園市,設立,公司登記
6,00008610,百璽國際有限公司,莊O湖,臺中市西區向上路一段36號6樓之3,100000,0,0,2025-03-01,臺中市,設立,公司登記
7,00008843,米水易設計創意影像有限公司,湯O君,臺中市豐原區社皮里社皮路76巷38號3樓之1,200000,0,0,2025-03-01,臺中市,設立,公司登記
8,00013634,瑔瀅企業有限公司,謝O洧,高雄市鳳山區大明路55巷5號,1000000,0,0,2025-03-01,高雄市,設立,公司登記
9,00015823,好室產業有限公司,張O碩,新北市新店區北新路2段262號3樓之3,100000,0,0,2025-03-01,新北市,設立,公司登記


In [25]:
# 抽樣條件
k = 100                              # 固定間隔
r = random.randint(0, k-1)
N = df_sys.shape[0]                  # 母體
n = N // k

if r < N % k:
    n += 1


In [26]:
# 每隔 k 抽 第 r 個位置
# 直到抽滿 N 
def sample_system(N:int, k:int, r:int) -> list:
    x = r
    sys = []

    while x < N:
        sys.append(x)
        x = x + k

    return sys

In [27]:
samp = sample_system(N,k,r)
n = len(samp)
print(n)
print(samp)

121
[68, 168, 268, 368, 468, 568, 668, 768, 868, 968, 1068, 1168, 1268, 1368, 1468, 1568, 1668, 1768, 1868, 1968, 2068, 2168, 2268, 2368, 2468, 2568, 2668, 2768, 2868, 2968, 3068, 3168, 3268, 3368, 3468, 3568, 3668, 3768, 3868, 3968, 4068, 4168, 4268, 4368, 4468, 4568, 4668, 4768, 4868, 4968, 5068, 5168, 5268, 5368, 5468, 5568, 5668, 5768, 5868, 5968, 6068, 6168, 6268, 6368, 6468, 6568, 6668, 6768, 6868, 6968, 7068, 7168, 7268, 7368, 7468, 7568, 7668, 7768, 7868, 7968, 8068, 8168, 8268, 8368, 8468, 8568, 8668, 8768, 8868, 8968, 9068, 9168, 9268, 9368, 9468, 9568, 9668, 9768, 9868, 9968, 10068, 10168, 10268, 10368, 10468, 10568, 10668, 10768, 10868, 10968, 11068, 11168, 11268, 11368, 11468, 11568, 11668, 11768, 11868, 11968, 12068]


In [28]:
df_sys["系統抽樣"] = df_sys.index.isin(samp)
df_sys["系統抽樣"].sum()

np.int64(121)

In [29]:
df_sys.iloc[r:r+10*k+1:k]

,統一編號,公司名稱,負責人,公司地址,資本總額,實收資本額,在境內營運資金,產製日期,縣市,狀態,類別,系統抽樣
68,00125384,米奧繁星股份有限公司,許O嘉,臺中市西屯區潮洋里朝富路213號20樓之5,1000000000,8928570,0,2025-03-01,臺中市,設立,公司登記,True
168,12461442,鈺翔廣告股份有限公司,李O北,臺北市中山區建國北路3段79號7樓,12000000,12000000,0,2025-03-01,臺北市,設立,公司登記,True
268,16319452,美福倉儲股份有限公司,李O波,桃園市蘆竹區長安路2段109號,1300000000,1300000000,0,2025-03-01,桃園市,設立,公司登記,True
368,22797902,勵德自動化有限公司,張O真,彰化縣大村鄉平和村中山路三段2巷430號,10000000,0,0,2025-03-01,彰化縣,設立,公司登記,True
468,24385342,喧嘩股份有限公司,劉O軒,新北市中和區中山路2段389號3樓,400000000,130000000,0,2025-03-01,新北市,設立,公司登記,True
568,24574195,佶翔國際貿易有限公司,陳O煌,新北市汐止區瑞松街206巷3號(1樓),5000000,0,0,2025-03-01,新北市,設立,公司登記,True
668,24678451,涼山休閒育樂有限公司,孫O城,屏東縣瑪家鄉凉山村凉山14之20號,30000000,0,0,2025-03-01,屏東縣,設立,公司登記,True
768,24757407,長增數位科技股份有限公司,林O怡,臺北市南港區昆陽街16號17樓,50000000,300000,0,2025-03-01,臺北市,設立,公司登記,True
868,24877965,技冠企業有限公司,王O評,雲林縣虎尾鎮安慶里民主四路56號1樓,1000000,0,0,2025-03-01,雲林縣,設立,公司登記,True
968,24960041,皇磊國際有限公司,曾O迪,臺北市信義區松德路10號7樓,5000000,0,0,2025-03-01,臺北市,設立,公司登記,True


In [30]:
df_sys['系統抽樣'] = df_sys['系統抽樣'].apply(lambda x: 'Y' if x else '')

In [31]:
# 抽樣統計
df_sys_sum = df_sys["縣市"].value_counts().reset_index(name="件數")
df_sys_samp = df_sys[df_sys["系統抽樣"]=="Y"]["縣市"].value_counts().reset_index(name="系統抽樣")

In [32]:
# 跟 df_sys_sum 合併
df_sys_sum = pd.merge(df_sys_sum, df_sys_samp, on="縣市", how="left")
df_sys_sum = df_sys_sum.fillna(0)
df_sys_sum["百分比"] = df_sys_sum["系統抽樣"] / df_sys_sum["件數"]
df_sys_sum

,縣市,件數,系統抽樣,百分比
0,臺北市,3623,38.0,0.010489
1,新北市,2241,22.0,0.009817
2,臺中市,1906,16.0,0.008395
3,高雄市,1384,17.0,0.012283
4,桃園市,850,7.0,0.008235
5,臺南市,538,6.0,0.011152
6,彰化縣,217,2.0,0.009217
7,新竹市,173,0.0,0.000000
8,新竹縣,171,1.0,0.005848
9,屏東縣,146,3.0,0.020548


In [33]:
samp_param = pd.DataFrame({'參數': ["間隔 k", "位置 r", "母體 N", "抽樣 n"], '值': [k, r, N, n]})
samp_param

,參數,值
0,間隔 k,100
1,位置 r,68
2,母體 N,12138
3,抽樣 n,121


In [34]:
# 儲存 Excel 
xlsx = "lab-25-系統抽樣.xlsx"

with pd.ExcelWriter(xlsx) as writer:
    df_sys.to_excel(writer, sheet_name='系統抽樣', index=True)
    samp_param.to_excel(writer, sheet_name='抽樣參數', index=False)
    df_sys_sum.to_excel(writer, sheet_name='抽樣統計', index=False)
    
    

### 分層抽樣 (stratified sampling) <a id="03"></a>

將抽樣單位按某種特徵或某種規則劃分為不同的層，然後從不同的層中獨立、隨機地抽取樣本。從而保證樣本的結構與母體的結構比較相近，從而提高估計的精度


In [35]:
# 以「縣市」 為特徵群組
# 每群抽 8 個，最多佔 50%

m = 22    # 統計共 22 個縣市
k = 8     # 每群抽幾個
f = 0.5   # 比例不超過 0.5 
n = 0     # 實際抽得筆數，預設為 0

In [36]:
def sample_stratified(data, k=1, f=0.5) -> list:
    cnt = data.shape[0]
    if k > cnt*f:
        # 抽比例
        samp = data.sample(frac=f)
    else:
        # 抽固定件數    
        samp = data.sample(n=k)

    return list(samp.index)

In [37]:
df_str = df1.copy().reset_index(drop=True)
df_str.head(5)

,統一編號,公司名稱,負責人,公司地址,資本總額,實收資本額,在境內營運資金,產製日期,縣市,狀態,類別
0,00001200,形韻科技股份有限公司,林O民,新竹市東區中華路二段198號1樓及地下一樓,1000000,1000000,0,2025-03-01,新竹市,設立,公司登記
1,00002799,鑫晶生物科技有限公司,陳O英,臺北市大同區市民大道1段209號10樓,1000000,0,0,2025-03-01,臺北市,設立,公司登記
2,00004810,道原智慧服務有限公司,廖O奇,新北市五股區中興路1段162號（2樓）,2000000,0,0,2025-03-01,新北市,設立,公司登記
3,00004945,顯樂文化事業有限公司,黃O宇,花蓮縣新城鄉新秀村光復路497號2樓之1,100000,0,0,2025-03-01,花蓮縣,設立,公司登記
4,00005938,瑞盛人力資源管理顧問有限公司,陳O瑞,臺北市信義區信義路4段415號5樓之4,100000,0,0,2025-03-01,臺北市,設立,公司登記


In [38]:
city = df_str["縣市"].unique()
df_str["縣市"].value_counts()

縣市
臺北市    3623
新北市    2241
臺中市    1906
高雄市    1384
桃園市     850
臺南市     538
彰化縣     217
新竹市     173
新竹縣     171
屏東縣     146
花蓮縣     120
基隆市     106
苗栗縣     106
雲林縣     102
南投縣      94
宜蘭縣      91
嘉義市      90
臺東縣      65
嘉義縣      61
金門縣      30
澎湖縣      12
連江縣       7
新竹科       5
Name: count, dtype: int64

In [39]:
# 抽樣參數
k = 6
m = len(city)
f = 0.5
N = df_str.shape[0]

In [40]:
samp = []
for c in city:
    samp.extend(sample_stratified(df_str[df_str["縣市"]==c], k, f))

In [41]:
n = len(samp)


In [42]:
df_str["分層抽樣"] = df_str.index.isin(samp)

In [43]:
df_str['分層抽樣'] = df_str['分層抽樣'].apply(lambda x: 'Y' if x else '')

In [44]:
# 抽樣統計
df_str_sum = df_str["縣市"].value_counts().reset_index(name="件數")
df_str_samp = df_str[df_str["分層抽樣"]=="Y"]["縣市"].value_counts().reset_index(name="分層抽樣")

In [45]:
# 跟 df_sys_sum 合併
df_str_sum = pd.merge(df_str_sum, df_str_samp, on="縣市", how="left")
df_str_sum = df_str_sum.fillna(0)
df_str_sum["百分比"] = df_str_sum["分層抽樣"] / df_str_sum["件數"]
df_str_sum

,縣市,件數,分層抽樣,百分比
0,臺北市,3623,6,0.001656
1,新北市,2241,6,0.002677
2,臺中市,1906,6,0.003148
3,高雄市,1384,6,0.004335
4,桃園市,850,6,0.007059
5,臺南市,538,6,0.011152
6,彰化縣,217,6,0.027650
7,新竹市,173,6,0.034682
8,新竹縣,171,6,0.035088
9,屏東縣,146,6,0.041096


In [46]:
samp_param = pd.DataFrame({'參數': ["特徵值", "單群件數 k", "母體 N", "抽樣 n", "佔比"], 
                           '值': ["縣市",k, N, n, round(n/N*100,2)]})
samp_param

,參數,值
0,特徵值,縣市
1,單群件數 k,6
2,母體 N,12138
3,抽樣 n,132
4,佔比,1.09


In [47]:
# 儲存 Excel 
xlsx = "lab-25-分層抽樣.xlsx"

with pd.ExcelWriter(xlsx) as writer:
    df_str.to_excel(writer, sheet_name='分層抽樣', index=False)
    samp_param.to_excel(writer, sheet_name='抽樣參數', index=False)
    df_str_sum.to_excel(writer, sheet_name='抽樣統計', index=False)
    

### 整群抽樣（cluster sampling）<a id="04"></a>
又稱群集抽樣，將母體中若干個單位合併為群，抽樣時直接抽取群，然後對中選群中的所有單位全部實施調查。抽樣時只需群的抽樣框，可簡化工作量，缺點是估計的精度較差。

In [48]:
# 以  資本總額 >= 100萬 為特徵群組


In [49]:
df1.head()

,統一編號,公司名稱,負責人,公司地址,資本總額,實收資本額,在境內營運資金,產製日期,縣市,狀態,類別
0,00001200,形韻科技股份有限公司,林O民,新竹市東區中華路二段198號1樓及地下一樓,1000000,1000000,0,2025-03-01,新竹市,設立,公司登記
1,00002799,鑫晶生物科技有限公司,陳O英,臺北市大同區市民大道1段209號10樓,1000000,0,0,2025-03-01,臺北市,設立,公司登記
2,00004810,道原智慧服務有限公司,廖O奇,新北市五股區中興路1段162號（2樓）,2000000,0,0,2025-03-01,新北市,設立,公司登記
3,00004945,顯樂文化事業有限公司,黃O宇,花蓮縣新城鄉新秀村光復路497號2樓之1,100000,0,0,2025-03-01,花蓮縣,設立,公司登記
4,00005938,瑞盛人力資源管理顧問有限公司,陳O瑞,臺北市信義區信義路4段415號5樓之4,100000,0,0,2025-03-01,臺北市,設立,公司登記


In [50]:
# 複製資料集
df_clu = df1.copy().reset_index(drop=True)
df_clu.head(30)


,統一編號,公司名稱,負責人,公司地址,資本總額,實收資本額,在境內營運資金,產製日期,縣市,狀態,類別
0,00001200,形韻科技股份有限公司,林O民,新竹市東區中華路二段198號1樓及地下一樓,1000000,1000000,0,2025-03-01,新竹市,設立,公司登記
1,00002799,鑫晶生物科技有限公司,陳O英,臺北市大同區市民大道1段209號10樓,1000000,0,0,2025-03-01,臺北市,設立,公司登記
2,00004810,道原智慧服務有限公司,廖O奇,新北市五股區中興路1段162號（2樓）,2000000,0,0,2025-03-01,新北市,設立,公司登記
3,00004945,顯樂文化事業有限公司,黃O宇,花蓮縣新城鄉新秀村光復路497號2樓之1,100000,0,0,2025-03-01,花蓮縣,設立,公司登記
4,00005938,瑞盛人力資源管理顧問有限公司,陳O瑞,臺北市信義區信義路4段415號5樓之4,100000,0,0,2025-03-01,臺北市,設立,公司登記
5,00006438,璟朝開發有限公司,李O煌,桃園市中壢區正光街175號(2樓),1000000,0,0,2025-03-01,桃園市,設立,公司登記
6,00008610,百璽國際有限公司,莊O湖,臺中市西區向上路一段36號6樓之3,100000,0,0,2025-03-01,臺中市,設立,公司登記
7,00008843,米水易設計創意影像有限公司,湯O君,臺中市豐原區社皮里社皮路76巷38號3樓之1,200000,0,0,2025-03-01,臺中市,設立,公司登記
8,00013634,瑔瀅企業有限公司,謝O洧,高雄市鳳山區大明路55巷5號,1000000,0,0,2025-03-01,高雄市,設立,公司登記
9,00015823,好室產業有限公司,張O碩,新北市新店區北新路2段262號3樓之3,100000,0,0,2025-03-01,新北市,設立,公司登記


In [51]:
capital = 1_000_000
df_clu["叢集"] = df1["資本總額"] >= capital

# 叢集筆數
N = df_clu["叢集"].sum()
N

5829

In [52]:
n = 120     # 實際抽 120 筆數

In [53]:
print(f"條件：資本總額 >= {capital:6,d}")
print(f"資料：\t{df1.shape[0]:6,d}")
print(f"叢集：\t{N:6,d}")
print(f"抽樣：\t{n:6,d}")
print(f"比例：\t{n/N:6,.2%}")


條件：資本總額 >= 1,000,000
資料：	12,138
叢集：	 5,829
抽樣：	   120
比例：	 2.06%


In [54]:
# TODO 
# 將 條件改為 >= 5_000_000

In [55]:
df_clu.head()

,統一編號,公司名稱,負責人,公司地址,資本總額,實收資本額,在境內營運資金,產製日期,縣市,狀態,類別,叢集
0,00001200,形韻科技股份有限公司,林O民,新竹市東區中華路二段198號1樓及地下一樓,1000000,1000000,0,2025-03-01,新竹市,設立,公司登記,True
1,00002799,鑫晶生物科技有限公司,陳O英,臺北市大同區市民大道1段209號10樓,1000000,0,0,2025-03-01,臺北市,設立,公司登記,True
2,00004810,道原智慧服務有限公司,廖O奇,新北市五股區中興路1段162號（2樓）,2000000,0,0,2025-03-01,新北市,設立,公司登記,True
3,00004945,顯樂文化事業有限公司,黃O宇,花蓮縣新城鄉新秀村光復路497號2樓之1,100000,0,0,2025-03-01,花蓮縣,設立,公司登記,False
4,00005938,瑞盛人力資源管理顧問有限公司,陳O瑞,臺北市信義區信義路4段415號5樓之4,100000,0,0,2025-03-01,臺北市,設立,公司登記,False


In [56]:
samp = df_clu[df_clu["叢集"]==True].sample(n=n).index

In [57]:
samp

Index([ 8723,  9678, 10734,  9029,  9074, 10168, 11414,  3526,  1284, 10890,
       ...
        6527,  1373,  8482,  4618,  4111,  1959,  8253,  5417, 11487,  9452],
      dtype='int64', length=120)

In [58]:
df_clu["叢集抽樣"] = df_clu.index.isin(samp)

In [59]:
df_clu['叢集'] = df_clu['叢集'].apply(lambda x: 'Y' if x else '')
df_clu['叢集抽樣'] = df_clu['叢集抽樣'].apply(lambda x: 'Y' if x else '')

In [60]:
# 抽樣統計
df_clu_sum = df_clu["縣市"].value_counts().reset_index(name="件數")
df_clu_grp = df_clu[df_clu["叢集"]=="Y"]["縣市"].value_counts().reset_index(name="叢集")
df_clu_samp = df_clu[df_clu["叢集抽樣"]=="Y"]["縣市"].value_counts().reset_index(name="叢集抽樣")

In [61]:
# 跟 df_clu_sum 合併
df_clu_sum = pd.merge(df_clu_sum, df_clu_grp, on="縣市", how="left")
df_clu_sum = df_clu_sum.fillna(0)

df_clu_sum = pd.merge(df_clu_sum, df_clu_samp, on="縣市", how="left")
df_clu_sum = df_clu_sum.fillna(0)

df_clu_sum["百分比"] = df_clu_sum["叢集抽樣"] / df_clu_sum["件數"]
df_clu_sum

,縣市,件數,叢集,叢集抽樣,百分比
0,臺北市,3623,3040,44.0,0.012145
1,新北市,2241,1870,14.0,0.006247
2,臺中市,1906,1581,18.0,0.009444
3,高雄市,1384,1158,14.0,0.010116
4,桃園市,850,717,8.0,0.009412
5,臺南市,538,434,5.0,0.009294
6,彰化縣,217,186,4.0,0.018433
7,新竹市,173,135,1.0,0.005780
8,新竹縣,171,134,4.0,0.023392
9,屏東縣,146,110,1.0,0.006849


In [62]:
samp_param = pd.DataFrame({'參數': [ "叢集條件","資料筆數","母體 N", "抽樣 n", "佔比"], 
                           '值': [f"資本總額 >= {capital:,d}", df1.shape[0],  N, n, round(n/N*100,2)]})
samp_param

,參數,值
0,叢集條件,"資本總額 >= 1,000,000"
1,資料筆數,12138
2,母體 N,5829
3,抽樣 n,120
4,佔比,2.06


In [63]:
# 儲存 Excel 
xlsx = "lab-25-叢集抽樣.xlsx"

with pd.ExcelWriter(xlsx) as writer:
    df_clu.to_excel(writer, sheet_name='叢集抽樣', index=False)
    samp_param.to_excel(writer, sheet_name='抽樣參數', index=False)
    df_clu_sum.to_excel(writer, sheet_name='抽樣統計', index=False)